
**Welcome to LoreaLab Repository!**

In this section, you can see and check our code. This code only prototyping to see the feasibilty of implementation and need detail works also using several API to connect with database, hardware, etc.
```
```



In [ ]:
import torch #For self training AI model
import clip #OpenAI Library for mix and match the image and text
from PIL import Image
import whisper #OpenAI Library for convert sound to text and work with AI
from transformers import pipeline
import streamlit as st
import openai
import speech_recognition as sr
from sklearn.feature_extraction.text import TfidfVectorizer #Convert text mood become "number" in math to be used as weighing in machine learning.
from sklearn.neighbors import NearestNeighbors #K-Nearest Neighbors to find similarity in machine learning
import random
import RPi.GPIO as GPIO  # Only if using Raspberry Pi


#Step 1: Create function to analyze image
def analyze_image(image_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    moods = ["calm", "romantic", "fresh", "warm", "luxurious", "playful"]
    text_tokens = clip.tokenize(moods).to(device)
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_tokens)
    similarity = (image_features @ text_features.T).softmax(dim=-1)
    best_mood = moods[similarity.argmax().item()]
    return generate_perfume_formula(best_mood)

#Step 2: Create function to analyze audio
def analyze_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    text = result["text"]
    sentiment_model = pipeline("sentiment-analysis")
    sentiment = sentiment_model(text)[0]["label"]
    return generate_perfume_formula(sentiment.lower())

#Step 3: Create function to detect user by proximity sensor using Raspberry pi
SENSOR_PIN = 4
GPIO.setmode(GPIO.BCM)
GPIO.setup(SENSOR_PIN, GPIO.IN)

def detect_proximity():
    """Detect when a user is near the sensor"""
    return GPIO.input(SENSOR_PIN) == 1  # Adjust based on sensor logic

#Step 4: Create function to analyze audio
#Dictionary for parfume formula
perfume_dict = {
    "calm": "80% lavender, 10% musk, 10% citrus",
    "romantic": "60% rose, 30% vanilla, 10% amber",
    "fresh": "50% lemon, 30% mint, 20% green tea",
    "warm": "40% sandalwood, 40% cinnamon, 20% vanilla",
    "luxurious": "50% oud, 30% jasmine, 20% saffron",
    "playful": "50% fruity, 30% caramel, 20% floral"
}


moods = list(perfume_dict.keys()) # Data for machine learning

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(moods)  #Vectorizer  NLP

knn = NearestNeighbors(n_neighbors=1, metric='cosine')
knn.fit(X) # KNN model to find the similarities mood

def generate_perfume_formula(mood):
    if mood in perfume_dict:
        return perfume_dict[mood]

    else:
        mood_vector = vectorizer.transform([mood]) # Transform input mood
        nearest_idx = knn.kneighbors(mood_vector, return_distance=False)[0][0]
        closest_mood = moods[nearest_idx]  # Find the nearest mood

        # Ambil komposisi dari mood terdekat
        base_formula = perfume_dict[closest_mood]

        # Generate variasi dengan mengganti beberapa bahan
        ingredients = base_formula.split(", ")
        random.shuffle(ingredients)  # Acak urutan
        new_formula = ", ".join(ingredients)

        return f"(Auto-generated) {new_formula} (Closest match: {closest_mood})"


#Step 5: Create function to define formula that needed to create
def choose_formula(concentration, volume, top, middle, base):
    return f"Concentration : {concentration}, Fill Option: {fill_option}, Volume: {volume}ml, Main Accord: {main_accord}, Top Notes: {top}, Middle Notes: {middle}, Base Notes: {base}"

#Step 6: Create function to create interactive voice chat based on question that we create
def interactive_voice_chat():
    questions = [
        "Do you prefer to build Eau de Parfume, Eau de Cologne, or Eau de Toilette?", #Question for concentration
        "Do you want to fill the parfum in new bottle or refill? If you want to refill, don't forget to put your bottle in refill place below", #Question for fill option
        "Do you want to fill in 30, 50, or 100ml?", #Question for volume
        "What you want for main accords?", #Question for accords
        "Do you prefer bergamot, freecia, or orange for top notes?", #Question for top notes
        "Do you prefer Rose, Jasmine, or pine for middle note?", #Question for middle notes
        "Do you prefer Amber, Mask, or Moses for base note?" #Question for base notes

    ]
    recognizer = sr.Recognizer()
    responses = []
    with sr.Microphone() as source:
        for question in questions:
            st.write(question)
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source)
            try:
                text = recognizer.recognize_google(audio)
                responses.append(text)
            except sr.UnknownValueError:
                responses.append("Unknown")
    return analyze_text_responses(responses)

#Step 7: Text response analyzer
def analyze_text_responses(responses):
    sentiment_model = pipeline("sentiment-analysis")
    combined_text = " ".join(responses)
    sentiment = sentiment_model(combined_text)[0]["label"].lower()
    return generate_perfume_formula(sentiment)

#Step 8: Create Function for Guide user to interact with our machine
st.title("Welcome to LoreaLab")
option = st.radio("Choose your prefer Method", ("Upload Image", "Select Formula", "Voice Interaction"))

if option == "Upload Image":
    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "png"])
    if uploaded_file is not None:
        formula = analyze_image(uploaded_file)
        st.write("Generated Perfume Formula:", formula)

elif option == "Select Formula":
    concentration = st.selectbox("Choose Concentration", ["Eau de Cologne", "Eau de Toilette", "Eau de Parfum"])
    volume = st.selectbox("Select Volume (ml)", 30, 50, 100)
    fill_option = st.selectbox("Fresh flask or a refill of perfection?", ["New Bottle", "Refill"])
    main_accord = st.selectbox("Main accords", ["Sweet", "Synthetic", "Spicy", "Woody", "Jasmine"])
    top_note = st.selectbox("Top Notes", ["Bergamot", "Freecia", "Orange", "Mint", "Dust"])
    middle_note = st.selectbox("Middle Notes", ["Hellotrope", "Rose", "Jasmine", "Pine", "Gardenia"])
    base_note = st.selectbox("Base Notes", ["Amber", "Mask", "Moses", "Vanila", "Magnolia"])
    if st.button("Generate Formula"):
        formula = choose_formula(concentration, volume, top_note, middle_note, base_note)
        st.write("Custom Perfume Formula:", formula)

elif option == "Voice Interaction":
    st.write("Move closer to the machine to start interaction"):
    if detect_proximity() # Automatically triggers when user is near the sensor
        formula = interactive_voice_chat()
        st.write("Generated Perfume Formula:", formula)
